In [1]:
from pyspark import SparkContext
import json
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

In [2]:
f= open('/yelp/yelp_academic_dataset_review.json')
print json.loads(f.readline())
f.close()

{u'votes': {u'funny': 0, u'useful': 0, u'cool': 0}, u'user_id': u'PUFPaY9KxDAcGqfsorJp3Q', u'review_id': u'Ya85v4eqdd6k9Od8HbQjyA', u'text': u'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.', u'business_id': u'5UmKMjUEUNdYWqANhGckJw', u'stars': 4, u'date': u'2012-08-01', u'type': u'review'}


In [3]:
sc = SparkContext()

In [4]:
lines = sc.textFile("/yelp/yelp_academic_dataset_review.json")

In [5]:
reviews = lines.map(lambda x: json.loads(x))

In [6]:
full_reviews = reviews.filter(
    lambda x: x['votes']['funny'] and x['votes']['useful'] and x['votes']['cool'])

In [7]:
good_or_bad_full_reviews = full_reviews.filter(lambda x: x['stars'] == 5 or x['stars'] == 1 )

In [8]:
parsedData = good_or_bad_full_reviews.map(
    lambda x: 
    LabeledPoint(
        1.0 if float(x['stars']) == 5 else 0.0, 
        [
            float(x['votes']['funny']),
            float(x['votes']['useful']),
            float(x['votes']['cool'])
        ]
    )
)

In [9]:
model = LogisticRegressionWithLBFGS.train(parsedData)

In [10]:
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.193882807924


In [11]:
model.predict([8.0,9.0,1.0])

0

In [12]:
model.predict([6.0,4.0,1.0])

0

In [13]:
model.predict([0,0,0])

0

In [14]:
model.predict([5.0,5.0,4.0])

1

In [15]:
model.predict([3.0,3.0,2.0])

1

In [16]:
model.predict([1.0,1.0,1.0])

1

In [17]:
model.predict([1.0,0.0,0.0])

0

In [18]:
model.predict([8.0,9.0,10.0])

1

In [19]:
model.predict([0,0,10.0])

1